In [10]:
import numpy as np
import math

front_tire_coeff_Fy = [0.349, -0.00115, 8.760, 730.300, 1745.322, 0.0139, -0.000277, 1.02025435, 0, 0, 0, 0, 0, 0, 0, 0.00362, -0.0143, -0.0116]

front_tire_coeff_Fx = [0.46024966176377113, 4000.509873697152, 1097.1712081460967, 202.18848632159495, 100.8812198037175, -0.2557010431649166, 0.3066955241461764, 0.011822770671297778, -1.9521015799737094, 0, 0, 0, 0, 0]

rear_tire_coeff_Fy = [1.384, -0.0003117, -2.936, 668.1, 1599, 0.03877, 0.0003177, 0.6252, 0, 0, 0, 0, 0, 0, 0, 0.005249, 0.0508, -0.1956]

rear_tire_coeff_Fx = [0.46024966176377113, 4000.509873697152, 1097.1712081460967, 202.18848632159495, 100.8812198037175, -0.2557010431649166, 0.3066955241461764, 0.011822770671297778, -1.9521015799737094, 0, 0, 0, 0, 0]

tire_scaling = 0.55

In [31]:

# Redo cornering stiffness implementation
def _get_comstock_forces(SR, SA, FZ, IA):
    if FZ <= 0.0:
        return np.array([0, 0, 0])
    else:
        FX = _long_pacejka(FZ, SR)
        FY = _lat_pacejka(IA, FZ, SA)

        Ca = (_long_pacejka(FZ, 0.005) - _long_pacejka(FZ, -0.005)) / (.01)
        Cs = (_lat_pacejka(IA, FZ, 0.005) - _lat_pacejka(IA, FZ, -0.005)) / (.01)
        FY_adj = com_lat(SA, SR, FX, FY, IA, FZ, Cs) 
        FX_adj = com_long(SA, SR, FX, FY, IA, FZ, Ca)

        if SA > 0:
            FY_adj = FY_adj * -1
        
        return np.array([FX_adj, FY_adj, FZ])

# Fits done in a messy way right now. Fix this later.
def _lat_pacejka(inclination_angle:float, normal_force:float, slip_angle:float):
    
    if normal_force == 0:
        return 0
    
    slip_degrees = slip_angle * 180 / math.pi # degrees
    inclination_degrees = inclination_angle * 180 / math.pi # degrees
    
    [a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12, a13, a14, a15, a16, a17] = front_tire_coeff_Fy
    C = a0
    D = normal_force * (a1 * normal_force + a2) * (1 - a15 * inclination_degrees ** 2)

    BCD = a3 * math.sin(math.atan(normal_force / a4) * 2) * (1 - a5 * abs(inclination_degrees))
    B = BCD / (C * D)

    H = a8 * normal_force + a9 + a10 * inclination_degrees
    E = (a6 * normal_force + a7) * (1 - (a16 * inclination_degrees + a17) * math.copysign(1, slip_degrees + H))
    V = a11 * normal_force + a12 + (a13 * normal_force + a14) * inclination_degrees * normal_force
    Bx1 = B * (slip_degrees + H)
    
    return tire_scaling * (D * math.sin(C * math.atan(Bx1 - E * (Bx1 - math.atan(Bx1)))) + V)

def _long_pacejka(normal_force:float, SR:float):
    if normal_force <= 0:
        return 0
    try:
        SR = SR * 100
        FZ = normal_force / 1000
        [b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13] = front_tire_coeff_Fx
        C = b0
        D = FZ * (b1 * FZ + b2)

        BCD = (b3 * FZ**2 + b4 * FZ) * np.exp(-1 * b5 * FZ)

        B = BCD / (C * D)

        H = b9 * FZ + b10

        E = (b6 * FZ**2 + b7 * FZ + b8) * (1 - b13 * np.sign(SR + H))

        V = b11 * FZ + b12
        Bx1 = B * (SR + H)
        
        return (D * np.sin(C * np.arctan(Bx1 - E * (Bx1 - np.arctan(Bx1)))) + V) * tire_scaling
    except:
        return 0
    
# Zero protection
def _zero_prot(SA, SR, IA, FZ, Cs, Ca, lat_long):
    epsi = 1 / 1e64
    SA_adj = SA + epsi
    SR_adj = SR + epsi
    FX_adj = _long_pacejka(FZ, SR_adj)
    FY_adj = _lat_pacejka(IA, FZ, SA_adj)
    if lat_long == "lat":
        calc = ((FX_adj * FY_adj) / np.sqrt(SR_adj**2 * FY_adj**2 + FX_adj**2 * (np.tan(SA_adj))**2)) * (np.sqrt((1 - SR_adj)**2 * (np.cos(SA_adj))**2 * FY_adj**2 + (np.sin(SA_adj))**2 * Cs**2) / (Cs * np.cos(SA_adj)))
        return calc
    else:
        calc = ((FX_adj * FY_adj) / np.sqrt(SR_adj**2 * FY_adj**2 + FX_adj**2 * (np.tan(SA_adj))**2)) * (np.sqrt(SR_adj**2 * Ca**2 + (1 - SR_adj)**2 * (np.cos(SA_adj))**2 * FX_adj**2) / Ca)
        return calc

# Long and lat formulas from Comstock
def com_lat(SA, SR, FX, FY, IA, FZ, Cs):
    if np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA))**2) == 0:
        calc = _zero_prot(SA, SR, IA, FZ, Cs, 0, "lat")
        return abs(calc) * -1 if SA < 0 else abs(calc)
    else:
        try:
            calc = ((FX * FY) / np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA))**2)) * (np.sqrt((1 - SR)**2 * (np.cos(SA))**2 * FY**2 + (np.sin(SA))**2 * Cs**2) / (Cs * np.cos(SA)))
            return abs(calc) * -1 if SA < 0 else abs(calc)
        except:
            calc = _zero_prot(SA, SR, IA, FZ, Cs, 0, "lat")
            return abs(calc) * -1 if SA < 0 else abs(calc)
    
def com_long(SA, SR, FX, FY, IA, FZ, Ca):
    if np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA))**2) == 0:
        calc = _zero_prot(SA, SR, IA, FZ, 0, Ca, "long")
        return abs(calc) * -1 if SA < 0 else calc
    else:
        try:
            calc = ((FX * FY) / np.sqrt(SR**2 * FY**2 + FX**2 * (np.tan(SA))**2)) * (np.sqrt(SR**2 * Ca**2 + (1 - SR)**2 * (np.cos(SA))**2 * FX**2) / Ca)
            return abs(calc) * -1 if SR < 0 else abs(calc)
        except:
            calc = _zero_prot(SA, SR, IA, FZ, 0, Ca, "long")
            return abs(calc) * -1 if SA < 0 else calc

In [33]:
print(_get_comstock_forces(0, 10, 1000, 1))

print(_get_comstock_forces(0, 10, 1000, 1))

[ 1.60758044e-60 -3.07518996e+04  1.00000000e+03]
[ 1.60758044e-60 -3.07518996e+04  1.00000000e+03]
